In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import os
import json
import copy
import joblib
import pandas as pd
import numpy as np
import time
from typing import Dict, Any, Optional, Union, List, Tuple, Type

from constants import DEVICE
from data_handling import DataHandler
from metrics import weighted_cross_entropy_loss
from mlp_model import objective_mlp

Using MPS device (Apple Silicon GPU)


In [2]:
dh = DataHandler()

DataHandler initialized - Using 114 features - Test year: 2020


In [3]:
# First, set up the ASHA pruner
asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
    min_resource=20,        # Minimum number of steps before pruning
    reduction_factor=2,    # Reduction factor for successive halving
    min_early_stopping_rate=0
)

In [4]:
study_mlp3 = optuna.create_study(direction="minimize", pruner=asha_pruner)

[I 2025-05-11 13:05:55,046] A new study created in memory with name: no-name-cadc6d1b-56c4-4056-85c7-3cfe252c6ca8


In [5]:
study_mlp3.optimize(
    lambda trial: objective_mlp(trial,
                            input_dim=dh.input_dim,
                            depth=3,
                            dataloaders=dh.get_nn_data('cv', batch_size=256),
                            max_epochs=128),
    n_trials=64,  # Number of trials to run
    timeout=600,   # Timeout in seconds
    n_jobs=-1,     # Use all available cores
)

[I 2025-05-11 13:07:38,724] Trial 6 pruned. 
[I 2025-05-11 13:07:43,569] Trial 4 pruned. 
[I 2025-05-11 13:09:05,130] Trial 3 pruned. 
[I 2025-05-11 13:09:25,158] Trial 9 pruned. 
[I 2025-05-11 13:10:40,102] Trial 10 pruned. 
[I 2025-05-11 13:12:14,364] Trial 12 pruned. 
[I 2025-05-11 13:12:44,183] Trial 11 pruned. 
[I 2025-05-11 13:15:15,765] Trial 13 pruned. 
[I 2025-05-11 13:15:22,190] Trial 7 finished with value: 1.0639335879912741 and parameters: {'learning_rate': 0.030404413973040313, 'weight_decay': 0.0004494607467880671, 'n_hidden_1': 16, 'dropout_rate_1': 0.17496197346951176, 'n_hidden_2': 80, 'dropout_rate_2': 0.4008359098210006, 'n_hidden_3': 56, 'dropout_rate_3': 0.37737783585462537}. Best is trial 7 with value: 1.0639335879912741.
[I 2025-05-11 13:15:28,508] Trial 1 finished with value: 1.054548246738238 and parameters: {'learning_rate': 0.006915658603149061, 'weight_decay': 1.6291229598189416e-05, 'n_hidden_1': 104, 'dropout_rate_1': 0.4564157994013289, 'n_hidden_2': 104,

In [6]:
# study_mlp3.optimize(
#     lambda trial: objective_mlp(trial,
#                             input_dim=dh.input_dim,
#                             depth=3,
#                             dataloaders=dh.get_nn_data('cv', batch_size=256),
#                             max_epochs=128),
#     n_trials=64,  # Number of trials to run
#     timeout=1800,   # Timeout in seconds
#     n_jobs=-1,     # Use all available cores
# )

In [7]:
# # Save the complete study to a file
# study_filename = f"mlp3_optuna_study_{time.strftime('%Y%m%d_%H%M%S')}.pkl"
# joblib.dump(study_mlp3, study_filename)
# print(f"Study saved to {study_filename}")